# Post process scripts for the video results
### This notebooks reads the video results frame by frame and combine them for the entire video
### In the future we need to modify this script to combine frames from foreground and background

In [20]:
# change these to your own paths .....
path_to_videos_input = "/home/vinker/dev/input_images/videos_input"
path_to_results = "/home/vinker/dev/background_project/experiements/obj_vit_demo/"
# the name of the video you want to process
video_filename = "horse_vid.mp4"
path_to_res_all = f"{path_to_results}/{os.path.splitext(video_filename)[0]}"
path_to_result_sketches = f"{path_to_res_all}/object"
assert os.path.exists(path_to_result_sketches)

path_to_save_res_frames = f"{path_to_res_all}/object_res_frames"
if not os.path.exists(path_to_save_res_frames):
    os.mkdir(path_to_save_res_frames)

In [18]:
import pydiffvg
import re
import os
import subprocess as sp
import moviepy.editor as mvp
from IPython.display import clear_output
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

## Utils

In [13]:
def read_svg(path_svg, multiply=False):
    device = torch.device("cuda" if (
        torch.cuda.is_available() and torch.cuda.device_count() > 0) else "cpu")
    canvas_width, canvas_height, shapes, shape_groups = pydiffvg.svg_to_scene(
        path_svg)
    if multiply:
        canvas_width *= 2
        canvas_height *= 2
        for path in shapes:
            path.points *= 2
            path.stroke_width *= 2
    _render = pydiffvg.RenderFunction.apply
    scene_args = pydiffvg.RenderFunction.serialize_scene(
        canvas_width, canvas_height, shapes, shape_groups)
    img = _render(canvas_width,  # width
                  canvas_height,  # height
                  2,   # num_samples_x
                  2,   # num_samples_y
                  0,   # seed
                  None,
                  *scene_args)
    img = img[:, :, 3:4] * img[:, :, :3] + \
        torch.ones(img.shape[0], img.shape[1], 3,
                   device=device) * (1 - img[:, :, 3:4])
    img = img[:, :, :3]
    return img


In [22]:
res_dirs = sorted(os.listdir(path_to_result_sketches))

for dir_ in res_dirs:
    cur_folder = f"{path_to_result_sketches}/{dir_}"
    svg_filename = [f for f in os.listdir(cur_folder) if ".svg" in f][0]
    sketch_res = read_svg(f"{cur_folder}/{svg_filename}", multiply=True).cpu().numpy()
    sketch = Image.fromarray((sketch_res * 255).astype('uint8'), 'RGB')
    sketch.save(f"{path_to_save_res_frames}/{dir_}.png")
    
sp.run(["ffmpeg", "-y", "-framerate", "10", "-pattern_type", "glob", "-i", 
            f"{path_to_save_res_frames}/*.png", "-vb", "20M", f"{path_to_res_all}/sketch_object.mp4"])
clear_output()
# # # display(img_)
display(mvp.ipython_display(f"{path_to_res_all}/sketch_object.mp4"))